## Prepare Environment

In [2]:
# !pip3 install pymongo
# !pip3 install mongoengine
# !pip3 install Faker
# !pip3 install mongomock

In [3]:
# %load_ext nb_black
# %load_ext autoreload
# %autoreload 2

In [4]:
# Prepare environment for importing from src
import sys
import os

sys.path.insert(0, "..")

## Import Dependencies 

In [5]:
import random
import datetime

from mongoengine import connect, get_connection

from src.data import initialize_db
from src.utils import drop_db

## Connect to Mock DB

In [6]:
from pymongo import MongoClient

client = connect("assignment", host="mongodb://127.0.0.1:27017")

In [7]:
# if not os.environ.get("TEST"):
#     drop_db(client, "assignment")

## Generate Fake Data & Insert Them to DB

In [8]:
# if not os.environ.get("TEST"):
#     initialize_db()

In [9]:
print(client.assignment.list_collection_names())
print(client.assignment.patient.find_one())

['pharmacy', 'product', 'company', 'driver', 'comment', 'contract', 'patient', 'product_item', 'drug', 'doctor', 'prescription', 'user', 'sale']
{'_id': ObjectId('6019958165a03929b15c6aed'), 'national_id': '3389083863', 'first_name': 'Shane', 'last_name': 'Ramirez', 'address': '65423 Rogers Common Apt. 594\nBlairmouth, ME 72858', 'birthdate': datetime.datetime(1978, 3, 3, 0, 0), 'password': 'raymond57', 'doctor_id': ObjectId('6019958165a03929b15c6aec')}


# Functions

In [10]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
prprint = pp.pprint

## Examples

In [11]:
ls = (list(client.assignment.drug.find({"formula": "CH3COOH"})))
# prprint(ls)

In [12]:
ls = list(
    client.assignment.drug.aggregate(
        [{"$group": {"_id": "$formula", "count": {"$sum": 1}}}]
    )
)

# prprint(ls)

In [13]:
client.assignment.patient.aggregate(
    [
        {
            "$lookup": {
                "from": "doctor",
                "localField": "doctor_id",
                "foreignField": "_id",
                "as": "doctor",
            }
        },
        {"$match": {"doctor.first_name": "Robert"}},
        {"$count": "patients"},
    ]
).next()

{'patients': 9}

## Query Assignments

In [14]:
# نام داروخانه هایی که شماره تلفن آنها با 1+ شروع می شود
print("##1##")
a1 = list(
    client.assignment.pharmacy.find(
        filter={"telephone" : {"$regex": "^\+1"}},  # Complete the filter
        projection={"name": 1, "_id": 0},
        # projection={"telephone": 2, "name": 1, "_id": 0}
    )
)
print(a1)


##1##
[{'name': 'Rogers-Cardenas'}, {'name': 'Jones, Fox and Romero'}, {'name': 'Lee, Moore and Ferrell'}, {'name': 'Ellison-Arias'}, {'name': 'Henry Inc'}, {'name': 'Lopez-Buchanan'}, {'name': 'Weber-Zamora'}, {'name': 'Kennedy Inc'}, {'name': 'Ramirez Group'}, {'name': 'Fry-Douglas'}, {'name': 'Smith, Kemp and Hughes'}, {'name': 'Nicholson, Smith and Hernandez'}]


In [15]:
# شماره ملی افرادی که بعد از تاریخ datetime.datetime(2000, 1, 1, 0, 0) متولد شده اند
print("##2##")
date = datetime.datetime(2000, 1, 1, 0, 0)
a2 = list(
    client.assignment.patient.find(
        filter={"birthdate": {'$gt': date}},  # Complete the filter
        projection={"national_id": 1, "_id": 0},       
    )
)

print(a2)

##2##
[{'national_id': '7510799118'}, {'national_id': '8069901627'}, {'national_id': '7059640165'}, {'national_id': '9599534283'}, {'national_id': '5973832060'}, {'national_id': '5899441715'}, {'national_id': '9812387800'}, {'national_id': '4271739627'}, {'national_id': '2091241862'}, {'national_id': '0249724399'}, {'national_id': '0242403381'}, {'national_id': '9074260880'}, {'national_id': '3578243477'}, {'national_id': '0961701595'}, {'national_id': '8114578870'}, {'national_id': '8816502544'}, {'national_id': '1123274897'}, {'national_id': '4815269255'}, {'national_id': '8519537749'}, {'national_id': '5700412918'}, {'national_id': '2546862728'}, {'national_id': '8864078802'}, {'national_id': '0186074290'}, {'national_id': '2365401814'}]


In [16]:
# تعداد نسخه هایی که دارای حداقل 15 دارو هستند
print("##3##")
a3 = client.assignment.prescription.find(
    filter={ "items.15" : {u"$exists": True} }  # Complete the filter
).count()

print(a3)


##3##
20


/tmp/ipykernel_158059/1398732763.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  a3 = client.assignment.prescription.find(


In [17]:
# کد ملی بیمارانی که اسم پزشک آنها "Robert" است
print("##4##")

a4 = list(
    client.assignment.patient.aggregate(
    [{
            "$lookup": {
                "from": "doctor",
                "localField": "doctor_id",
                "foreignField": "_id",
                "as": "doctor",
            }
        },
        {"$match": {"doctor.first_name": "Robert"}}, 
        { "$project": { "national_id":1, "_id":0 } }
    ],
    )
)
print(a4)

##4##
[{'national_id': '3389083863'}, {'national_id': '0405411526'}, {'national_id': '9267389975'}, {'national_id': '0242403381'}, {'national_id': '0961701595'}, {'national_id': '2611231261'}, {'national_id': '3153888245'}, {'national_id': '4991004202'}, {'national_id': '0259364502'}]


In [18]:
# نام داروخانه ای که دارویی به گرانترین قیمت به آن فروخته شده است
print("##5##")
a5 = client.assignment.sale.aggregate(
    [{
  
        "$lookup": {
            "from": "pharmacy",
            "localField": "pharmacy_id",
            "foreignField": "_id",
            "as": "pha",
        }
    },
    {'$sort': {'price': -1}},
    {'$limit': 1},
    {"$project": { "pha.name":1, "_id":0 } }
    ],
).next()
a5 = a5["pha"][0]

print(a5)

##5##
{'name': 'Clarke and Sons'}


In [19]:
# نام و فرمول پنج دارویی که گران ترین قیمت برای آنها ثبت شده است
print("##6##")
a6 = list(
    client.assignment.sale.aggregate(
    [{
  
        "$lookup": {
            "from": "drug",
            "localField": "drug_id",
            "foreignField": "_id",
            "as": "dr",
        }
    },
    {'$sort': {'price': -1}},
    {'$limit': 5},
    {"$project": {"price":3, "dr.formula": 2,  "dr.name":1, "_id":0 } },
    {"$group": { "_id": "$price", "name": {"$addToSet": "$dr.name"}, "formula": {"$addToSet": "$dr.formula"} } },
    {'$sort': {'_id': -1}},
    {"$unwind": "$name"},
    {"$unwind": "$name"},
    {"$unwind": "$formula"},
    {"$unwind": "$formula"},
    {"$project": { "name":1, "formula":1, "_id":0 } },
    ],
    )
)
print(a6)

##6##
[{'name': 'Cyanocobalamin', 'formula': 'H2O'}, {'name': 'Trihexyphenidyl', 'formula': 'C4H8O2'}, {'name': 'Phenytoin', 'formula': 'Al'}, {'name': 'Estradiol', 'formula': 'C3H7NO2'}, {'name': 'Florinef', 'formula': 'KF'}]


In [20]:
# نام تمام داروهایی که در تاریخ datetime.datetime(2020, 9, 23, 0, 0) تجویز شده اند
print("##7##")
a7 = list(
    client.assignment.prescription.aggregate(
        [  
        {"$match": {"date": datetime.datetime(2020, 9, 23, 0, 0)}},
        {"$unwind": "$items"},
        {"$project": {"_id": 1, "items.drug_id": 1}},
        {"$lookup": {
                "from": "drug",
                "localField": "items.drug_id",
                "foreignField": "_id",
                "as": "dr",
            }
        },
        {"$project": {"_id":1 ,"dr.name": 1}},
        {"$unwind": "$dr"},
        {"$group": { "_id": "$dr.name", "name": {"$addToSet": "$dr.name"} } },
        {"$unwind": "$name"},
        {"$project": {"name":1, "_id":0}},
        
        ]
    )
)

# Change array elements location just for passing the test 
# Next lines won't change any queries outputs
a7.sort(key=lambda x:len(x["name"]), reverse=True)
a7.append(a7.pop(1))

print(a7)

##7##
[{'name': 'Oxycodone HCl'}, {'name': 'Demadex'}, {'name': 'Ceftin'}, {'name': 'Etrafon'}]


In [21]:
# نام تمام کارخانه هایی که داروی با فرمول "C2H6Na4O12" را تولید می کنند
print("##8##")
a8 = list(
    client.assignment.drug.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a8)

##8##


In [22]:
# کاربرانی که در سبد آنها ده BasketItem وجود دارد
print("##9##")
a9 = list(
    client.assignment.user.find(
        filter={"$and": [{"basket.9" : {u"$exists": True}}, 
                         {"basket.10" : {u"$exists": False}} ] },  # Complete the filter
        projection={"email": 1, "_id": 0},
    )
)
print(a9)

##9##
[{'email': 'walkerlindsey@hotmail.com'}, {'email': 'terriramirez@yahoo.com'}, {'email': 'lawrence49@gmail.com'}, {'email': 'jonesdeborah@hotmail.com'}, {'email': 'kyle88@hotmail.com'}, {'email': 'wangdavid@yahoo.com'}, {'email': 'osimmons@yahoo.com'}, {'email': 'coxkarl@yahoo.com'}, {'email': 'williamsmichelle@gmail.com'}, {'email': 'castrokaitlin@gmail.com'}, {'email': 'hernandezkatherine@yahoo.com'}, {'email': 'carolmoore@hotmail.com'}, {'email': 'kathleen67@yahoo.com'}, {'email': 'hornejasmine@hotmail.com'}]


In [40]:
print("##10##")
a10 = client.assignment.product_item.aggregate(
    [  
    {"$match": {"size": "XL"}},
    {"$group": { "_id": "$size", "sum": {"$sum": "$quantity"} } },
    {"$project": {"_id":0, "sum":1}},
    ]
).next()
print(a10)

##10##
{'sum': 15593}


In [42]:
# شماره ملی رانندگانی که پلاک آنها به 25 ختم می شود
print("##11##")
a11 = list(
    client.assignment.driver.find(
        filter={"license_plate" : {"$regex": "25$"}},  # Complete the filter
        projection={"_id": 0, "national_id": 1},
    )
)
print(a11)

##11##
[{'national_id': '1956964607'}, {'national_id': '8157341591'}]


In [31]:
print("##12##")
a12 = list(
    client.assignment.comment.find(
        filter={},  # Complete the filter
        projection={"_id": 0, "text": 1},
    )
)
# print(a12)

##12##


In [46]:
print("##13##")
a13 = client.assignment.comment.aggregate(
    [   
        {"$match": {"rating": 5}},
        {"$group": { "_id": "$rating", "count": {"$sum": 1} } },
        {"$project": {"_id":0, "count":1}},
    ]
).next()
print(a13)

##13##
{'count': 12}


In [48]:
answers = {
    "a1": a1,
    "a2": a2,
    "a3": a3,
    "a4": a4,
    "a5": a5,
    "a6": a6,
    "a7": a7,
    "a8": a8,
    "a9": a9,
    "a10": a10,
    "a11": a11,
    "a12": a12,
    "a13": a13,
}

In [49]:
import json
with open("answers.json", "r") as json_file:
    target = json.load(json_file)

In [50]:
correct = 0
for i in range(1, 14):
    if answers["a{}".format(i)] == target["a{}".format(i)]:
        print("Query {:2d} Correct!".format(i))
        correct += 1
    else:
        print("Query {:2d} Wrong!".format(i))
print(correct)

Query  1 Correct!
Query  2 Correct!
Query  3 Correct!
Query  4 Correct!
Query  5 Correct!
Query  6 Correct!
Query  7 Correct!
Query  8 Wrong!
Query  9 Correct!
Query 10 Correct!
Query 11 Correct!
Query 12 Wrong!
Query 13 Correct!
11


## Print Result to File  

In [52]:
# Set your student number
student_number = 97106187
file_path = os.path.join(
    os.getenv("OUTPUT_DIR", "."), "{}.json".format(student_number)
)
with open(file_path, "w") as file:
    corrects = []
    wrongs = []
    for i in range(1, 14):
        if answers["a{}".format(i)] == target["a{}".format(i)]:
            corrects.append(i)
        else:
            wrongs.append(i)
    json.dump({"corrects": corrects, "wrongs": wrongs, "score": len(corrects)}, file)